In [90]:
class Layer:
    def __init__(self) -> None:
        self.input=None
        self.outpt=None

    def forward_propagation(self,input):
        raise NotImplementedError
    def backPropagation(self,output_error,learning_rate):
        raise NotImplementedError

In [91]:
import numpy as np
class FCLayer(Layer):
    def __init__(self,input_size,output_size) -> None:
        self.weights=np.random.rand(input_size,output_size)-.5
        self.bias=np.random.rand(1,output_size)-.5
    def forward_propagation(self, input):
        self.input=input
        self.outpt=np.dot(self.input,self.weights)+self.bias
        return self.outpt
    def backPropagation(self, output_error, learning_rate):
        dEdX=np.dot(output_error,self.weights.T)
        dEdW=np.dot(self.input.T,output_error)
        self.weights-=learning_rate*dEdW
        self.bias-=learning_rate*output_error
        return dEdX
        
    

In [92]:
class ActivationLayer(Layer):
    def __init__(self,activation,activation_prime) -> None:
        self.activation=activation
        self.activation_prime=activation_prime
    def forward_propagation(self, input):
        self.input=input
        self.output=self.activation(self.input)
        return self.output
    def backPropagation(self, output_error, learning_rate):
        return self.activation_prime(self.input)*output_error

In [178]:
def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1-np.tanh(x)**2

def sigmoid(x):
    return np.exp(x)

def sigmoid_prime(x):
    return sigmoid(x)*(1-sigmoid(x))

def atan(x):
    return np.arctan(x)
def atan_prime(x):
    return 1/(x**2+1)

In [179]:
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size

In [180]:
class NeuralNetwork:
    def __init__(self,loss,lossPrime):
        self.layers:list[Layer]=[]
        self.loss =loss
        self.lossPrime=lossPrime
    def addLayer(self,layer:Layer):
        self.layers.append(layer)
        

    def train(self,x_train,y_train,epochs=40,learning_rate=0.1):
        samples=len(x_train)
        for n in range(epochs):
            currentError=0
            for j in range(samples):
                output=x_train[j]
                for layer in self.layers:
                    output=layer.forward_propagation(output)
                currentError+=self.loss(y_train[j],output)

                outputError=self.lossPrime(y_train[j],output)
                for layer in reversed(self.layers):
                    outputError=layer.backPropagation(outputError,learning_rate)
            currentError/=samples
            print(f'epoch= {n} error={currentError}')

    def predict(self,x_test):
        samples=len(x_test)
        res=[]
        for n in range(samples):
            output=x_test[n]
            for layer in self.layers:
                output=layer.forward_propagation(output)
            res.append(output)
        return res
    


In [181]:
network=NeuralNetwork(mse,mse_prime)
network.addLayer(FCLayer(3,5))
network.addLayer(ActivationLayer(tanh,tanh_prime))
network.addLayer(FCLayer(5,7))
network.addLayer(ActivationLayer(tanh,tanh_prime))
network.addLayer(FCLayer(7,9))
network.addLayer(ActivationLayer(tanh,tanh_prime))
network.addLayer(FCLayer(9,8))
network.addLayer(ActivationLayer(atan,atan_prime))
network.addLayer(FCLayer(8,6))
network.addLayer(ActivationLayer(tanh,tanh_prime))
network.addLayer(FCLayer(6,3))
network.addLayer(ActivationLayer(tanh,tanh_prime))

In [182]:
x_train = np.array([[[0,0,0]], [[0,1,0]], [[1,0,1]], [[1,1,0]],[[0,1,1]],[[0,0,1]]])
y_train = np.array([[[0,0,1]], [[0,1,1]], [[1,1,0]], [[1,1,1]],[[1,0,0]],[[0,1,0]]])

In [190]:
network.train(x_train,y_train,epochs=6000,learning_rate=0.2)

epoch= 0 error=5.994851472395857e-05
epoch= 1 error=0.00025466670462091496
epoch= 2 error=0.002604853289499009
epoch= 3 error=0.01018500279568025
epoch= 4 error=0.05132557247792648
epoch= 5 error=0.2065718312578897
epoch= 6 error=0.1247385881153602
epoch= 7 error=0.08249912450331483
epoch= 8 error=0.030773021395037504
epoch= 9 error=0.04323809096309736
epoch= 10 error=0.03613577527197062
epoch= 11 error=0.024806813473497932
epoch= 12 error=0.042726333541567164
epoch= 13 error=0.024167409980417016
epoch= 14 error=0.03687323390354385
epoch= 15 error=0.03173588249998437
epoch= 16 error=0.009046748628480364
epoch= 17 error=0.007368555027912938
epoch= 18 error=0.0023696795112829005
epoch= 19 error=0.0011518471571675193
epoch= 20 error=0.0008696452310781094
epoch= 21 error=0.0009155199633246367
epoch= 22 error=0.0004154486759116668
epoch= 23 error=0.0005404162162016868
epoch= 24 error=0.0007972172260511066
epoch= 25 error=0.0009555080089854663
epoch= 26 error=0.0015101375137184108
epoch= 27 

In [191]:
network.predict([[1,0,0]])

[array([[0.12963504, 0.21459103, 0.99966098]])]